In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd

import tensorflow as tf

In [2]:
#추가부분
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')
#구글 드라이브 마운트
os.chdir("/content/drive/My Drive/data")
print(os.getcwd())

Mounted at /content/drive
/content/drive/My Drive/data


In [21]:
mode = 'test'
train_directory = 'public/{}/'.format(mode)
output_directory = 'public/{}_tfrec/'.format(mode)

num_shards = 1
generate_train_validation_splits = False
validation_split_size = 0.2     ## validation 비율

_FILE_IDS_KEY = 'file_ids'
_IMAGE_PATHS_KEY = 'image_paths'
_LABELS_KEY = 'labels'
_TEST_SPLIT = 'test'
_TRAIN_SPLIT = 'train'
_VALIDATION_SPLIT = 'validation'

In [22]:
id_dict = {}
with open('public/category.json') as f:
    json_data = json.load(f)
    for data in json_data['categories']:
        id_dict[data['landmark_name']] = data['landmark_id']

id_dict['NC백화점[강서]']

791

In [23]:
if generate_train_validation_splits:
    if validation_split_size <= 0 or validation_split_size >= 1:
      raise ValueError('Invalid VALIDATION split size. Expected inside (0,1)'
                       'but received %f.' % validation_split_size)

In [36]:
image_paths = glob.glob(os.path.join(train_directory, '*.jpg'), recursive=True)

print(image_paths)
file_ids = [os.path.basename(os.path.normpath(f))[:-4] for f in image_paths]

print(id_dict)

if mode == 'train':
    #labels = [f.split('/')[-2]for f in image_paths]
    labels = [id_dict[image_paths.split('/')[-2]] for f in image_paths]
elif mode == 'test':
    labels = []
else:
    raise ValueError('Unsupported dataset split name: %s' % name)

print('total image paths: {}'.format(len(image_paths)))

for i in range(11):
  print('file_ids example: {}'.format(file_ids[i]))
if mode == 'train':
    print('labels example: {}'.format(labels[0]))

['public/test/ff.jpg', 'public/test/ee.jpg', 'public/test/dd.jpg', 'public/test/cc.jpg', 'public/test/bb.jpg', 'public/test/aa.jpg', 'public/test/kk.jpg', 'public/test/jj.jpg', 'public/test/ii.jpg', 'public/test/hh.jpg', 'public/test/gg.jpg']
{'금천구청': 0, '두리랜드': 1, '이대원 장군묘': 2, '성공회대학교': 3, '송상현공 동상': 4, '진위향교': 5, '추계예술대학교': 6, '계명대학교 행소박물관': 7, '태조 이성계상': 8, '마포 농수산물 시장': 9, '대구서구문화회관': 10, '쉐라톤 서울 디큐브 시티 호텔': 11, '구리시 교문도서관': 12, '의정부시 상설 야외무대': 13, '연지어린이공원': 14, '종로성당': 15, '대화 레포츠공원 체육시설': 16, '하나로장로교회': 17, '서울대학교 관악캠퍼스': 18, '서울광장': 19, '영진종합사회복지관': 20, '한국항공대학교 우주박물관': 21, '좌동재래시장': 22, 'NC백화점 안산고잔점': 23, '청계천발원지': 24, '김포 독립운동 기념관': 25, '명동성당': 26, '경춘선숲길': 27, '유니온타워': 28, '황희선생 유적지': 29, '신포국제시장': 30, '안중근의사 기념관': 31, '의정부 정보도서관': 32, '인천 계양도서관': 33, '와우정사': 34, '덕소 도서관': 35, '은평역사한옥박물관': 36, '한중원 쉼터': 37, '무등경기장 야구장': 38, '한국지역난방공사 경기지사': 39, '절두산 순교 성지': 40, '안양문화 예술재단': 41, '문묘': 42, '뚝섬한강공원': 43, '관음정사': 44, '농업 박물관': 45, '육탄 이희복용사 동상': 46, '망원시장': 47, '의정부 성당': 48, '연

In [14]:
def _process_image(filename):
    # Read the image file.
    with tf.io.gfile.GFile(filename, 'rb') as f:
        image_data = f.read()

    # Decode the RGB JPEG.
    image = tf.io.decode_jpeg(image_data, channels=3)

    # Check that image converted to RGB
    if len(image.shape) != 3:
        raise ValueError('The parsed image number of dimensions is not 3 but %d' % (image.shape))
    height = image.shape[0]
    width = image.shape[1]
    if image.shape[2] != 3:
        raise ValueError('The parsed image channels is not 3 but %d' % (image.shape[2]))

    return image_data, height, width

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _convert_to_example(file_id, image_buffer, height, width, label=None):
    colorspace = 'RGB'
    channels = 3
    image_format = 'JPEG'
    features = {
      'image/height': _int64_feature(height),
      'image/width': _int64_feature(width),
      'image/colorspace': _bytes_feature(colorspace.encode('utf-8')),
      'image/channels': _int64_feature(channels),
      'image/format': _bytes_feature(image_format.encode('utf-8')),
      'image/id': _bytes_feature(file_id.encode('utf-8')),
      'image/encoded': _bytes_feature(image_buffer)
    }
    if label is not None:
        features['image/class/label'] = _int64_feature(label)
    example = tf.train.Example(features=tf.train.Features(feature=features))

    return example

def _write_tfrecord(output_prefix, image_paths, file_ids, labels):
  
    if output_prefix == _TEST_SPLIT:
        labels = [None] * len(image_paths)
    if not len(image_paths) == len(file_ids) == len(labels):
        raise ValueError('length of image_paths, file_ids, labels shoud be the' +
                         ' same. But they are %d, %d, %d, respectively' %
                         (len(image_paths), len(file_ids), len(labels)))

    spacing = np.linspace(0, len(image_paths), num_shards + 1, dtype=np.int)

    for shard in range(num_shards):
        output_file = os.path.join(output_directory,
                                   '%s-%.5d-of-%.5d' % (output_prefix, shard, num_shards))
        writer = tf.io.TFRecordWriter(output_file)
        print('Processing shard ', shard, ' and writing file ', output_file)
        for i in range(spacing[shard], spacing[shard + 1]):
            image_buffer, height, width = _process_image(image_paths[i])
            example = _convert_to_example(file_ids[i], image_buffer, height, width,
                                        labels[i])
            writer.write(example.SerializeToString())
        writer.close()

os.makedirs(output_directory, exist_ok=True)
        
if generate_train_validation_splits:
    def _shuffle_by_columns(np_array, random_state):
        columns = np_array.shape[1]
        columns_indices = np.arange(columns)
        random_state.shuffle(columns_indices)
        return np_array[:, columns_indices]
    
    total_images = len(file_ids)
    if not (len(image_paths) == total_images and len(labels) == total_images):
        raise ValueError('Inconsistencies between number of file_ids (%d), number '
                         'of image_paths (%d) and number of labels (%d). Cannot'
                         'shuffle the train dataset.'% (total_images,
                                                        len(image_paths),
                                                        len(labels)))

    labels_str = [str(label) for label in labels]
    image_attrs = np.stack((image_paths, file_ids, labels_str))
    image_attrs_idx_by_label = {}
    for index, label in enumerate(labels):
        if label not in image_attrs_idx_by_label:
            image_attrs_idx_by_label[label] = []
        image_attrs_idx_by_label[label].append(index)

    splits = {
        _VALIDATION_SPLIT: [],
        _TRAIN_SPLIT: []
    }
    
    rs = np.random.RandomState(np.random.MT19937(np.random.SeedSequence(0)))
    for label, indexes in image_attrs_idx_by_label.items():

        image_attrs_label = image_attrs[:, indexes]

        image_attrs_label = _shuffle_by_columns(image_attrs_label, rs)

        images_per_label = image_attrs_label.shape[1]
        cutoff_idx = max(1, int(validation_split_size * images_per_label))
        splits[_VALIDATION_SPLIT].append(image_attrs_label[:, 0 : cutoff_idx])
        splits[_TRAIN_SPLIT].append(image_attrs_label[:, cutoff_idx : ])

    validation_split = _shuffle_by_columns(
      np.concatenate(splits[_VALIDATION_SPLIT], axis=1), rs)
    train_split = _shuffle_by_columns(
      np.concatenate(splits[_TRAIN_SPLIT], axis=1), rs)

    validation_split = {
          _IMAGE_PATHS_KEY: validation_split[0, :].tolist(),
          _FILE_IDS_KEY: validation_split[1, :].tolist(),
          _LABELS_KEY: [int(label) for label in validation_split[2, :].tolist()]
    }
    train_split = {
          _IMAGE_PATHS_KEY: train_split[0, :].tolist(),
          _FILE_IDS_KEY: train_split[1, :].tolist(),
          _LABELS_KEY: [int(label) for label in train_split[2, :].tolist()]
    }
    
    _write_tfrecord(_VALIDATION_SPLIT,
                    validation_split[_IMAGE_PATHS_KEY],
                    validation_split[_FILE_IDS_KEY],
                    validation_split[_LABELS_KEY])
    _write_tfrecord(_TRAIN_SPLIT,
                    train_split[_IMAGE_PATHS_KEY],
                    train_split[_FILE_IDS_KEY],
                    train_split[_LABELS_KEY])
    
else:
    _write_tfrecord(mode, image_paths, file_ids, labels)   

Processing shard  0  and writing file  public/test_tfrec/test-00000-of-00001
